# Tarea 1
### Curso MA4702 año 2021.  Profesores Iván Rapaport y José Soto 

## Instrucciones

Esta tarea se debe realizar de manera **individual**.

El plazo de esta tarea es el día **siguiente** al laboratorio 1, es decir, el jueves 29 de abril a las 23:59

Debe entregar por ucursos un solo archivo Tarea.ipynb, al igual que en el laboratorio de prueba este archivo debe entregarse ejecutado por completo usando (Kernel-: Restart and Run All) y luego salvado al disco. El resultado de dicha ejecución es lo que debe entregar.


En esta tarea usted necesitará los paquetes siguientes
```
JuMP, Gurobi, Distances, Plots, StatsBase, DataFrames
```
(instálelos si no los tiene)


Instrucciones adicionales
1. Complete la celda de identificación
2. Resuelva los ejercicios 1 al 4
3. Una vez termine su tarea, ejecute el archivo jupyter completo usando (Kernel: Restart and Run All), o bien usando (Kernel: Restart and Clean Output) y luego ejecutando manualmente las celdas una a una en orden.
4. Fuarde el archivo resultado en un .ipynb. Ese es el archivo que debe entregar por ucursos.

NOTA IMPORTANTE:
Lea la tarea completa y trabaje con tiempo. Solo ejecutar las celdas ya completas le podría tomar varias decenas de minutos de cálculo.


# IDENTIFICACIÓN E INTRODUCCIÓN 

In [ ]:
# Celda de identificación. tipee y ejecute esta celda (como código, no como Markdown)
Mi_Nombre = " " # Escriba su nombre completo
@show(Mi_Nombre)
@show(Sys.BINDIR)
versioninfo()

In [ ]:
#paquetes a usar
using JuMP, Gurobi, Distances, Plots, StatsBase, DataFrames #instale estos paquetes antes si no los tiene en su sistema.

Calcularemos emparejamientos mínimos de conjuntos de puntos en el plano.
Consideremos $N$ puntos en el plano (con $N$ par), determinados por su vector ```coordx``` de coordenadas x y su vector ```coordy``` de  coordenadas y.

La función ```distancias``` de la siguiente celda  calcula una matriz $w$ con las distancias entre los puntos  guardados en una matriz triangular superior, es decir $w[i,j]$ es la distancia entre el punto $i$ y el punto $j$  para todo $1\leq i<j\leq N$.

Los modelos devolverán vectores indexados por aristas que no son necesariamente enteros, por lo cual la función ```dibuja``` de la siguiente celda será util para dibujar y analizarlos. 

In [ ]:
function distancias(coordx, coordy)
    N=length(coordx)
    w=zeros(N,N)
    for i in 1:N, j in i+1:N
        w[i,j]=euclidean([coordx[i],coordy[i]],[coordx[j],coordy[j]])
    end
    return w
end

function dibuja(coordx,coordy,arcos)
## Recibe dos arreglos de N valores donde (coordx[i],coordy[i]) son las coordenadas de la ciudad i
## Recibe además una matriz arcos de N x N, donde arcos[i,j] es el peso del arco [i,j]
## Dibuja los N puntos en el plano y dibuja los arcos con ancho de linea proporcional al peso.
    N=length(coordx)
    scatter(coordx,coordy,txt=text.(1:N,10,:bottom))
    for k in findall(!iszero, arcos)
        plot!([coordx[k[1]],coordx[k[2]]],[coordy[k[1]],coordy[k[2]]],arrow = false,lc=:blue, linewidth = 2*arcos[k[1],k[2]])
    end
    display(plot!(leg=false))
end


In [ ]:
#Un ejemplo con 20 puntos aleatorios.
#Se dibujan las aristas (1,7),(7,13),(5,13),(1,5) con un peso de 1
# y las aristas (2,8), (2,10), (8,10) con un peso de 0.5


N=20
a = sample(1:200, 20, replace = false)
b = sample(1:200, 20, replace = false)
w = distancias(a,b)
scatter!(a,b)
ciclo_ejemplo=zeros(N,N)
ciclo_ejemplo[1,7]=1
ciclo_ejemplo[7,13]=1
ciclo_ejemplo[5,13]=1
ciclo_ejemplo[1,5]=1

ciclo_ejemplo[2,10]=0.5
ciclo_ejemplo[2,8]=0.5
ciclo_ejemplo[8,10]=0.5

dibuja(a,b,ciclo_ejemplo)

In [ ]:
#imprimir la matriz de distancia w
w

***
***

En lo que sigue  $G=(V,E)$ es un grafo **completo** con $V=[N]$, Los siguientes cuatro modelos son modelos exactos (posiblemente extendidos y en el sentido de puntos extremos) para encontrar un emparejamiento de peso mínimo. Consideraremos $E=\{[i,j]\colon 1\leq i < j\leq N\}$.


\begin{align*}
\text{(M1)}\quad  \min \sum_{e\in E} x_ew_e\\
x(E) &= N/2\\
x_{e}+x_{f}&\leq 1, \forall e, f \in E\colon |e\cap f|=1\\
0\leq x_e&\leq 1, \forall e\in E\\
x_e&\in \mathbb{Z}, \forall e\in E\\
\mbox{}\\
\text{(M2)}\quad  \min \sum_{e\in E} x_ew_e\\
x(\delta(i))&= 1, \forall i\in V\\
x_e&\geq 0, \forall e\in E\\
x_e&\in \mathbb{Z}, \forall e\in E\\
\mbox{}\\
\text{(M3)}\quad  \min \sum_{e\in E} x_ew_e\\
x(\delta(i))&= 1, \forall i\in V\\
y(V)&=n/2\\
x_{ij}&\leq y_{i}+y_{j}, \forall e=ij \in E\\
x_{ij}&\leq 2-y_{i}+y_{j}, \forall e=ij \in E\\
x_e&\geq 0, \forall e\in E\\
0\leq y_v&\leq 1, \forall v\in V\\
y_v&\in \mathbb{Z}, \forall v\in V\\
\mbox{}\\
\text{(M4)}\quad  \min \sum_{e\in E} x_ew_e\\
x(E) &= N/2\\
x(E(S))&\leq \frac{|S|-1}{2}, \forall S\subseteq V, |S| \text{ impar}\\
0\leq x_e&\leq 1, \forall e\in E\\
\end{align*}

Los 4 modelos son bastante distintos entre sí. 

El primer modelo es un PLE con $m=\Theta(N^2)$ variables enteras y $\Theta(N^3)$ restricciones.

El segundo modelo es un PLE con $m=\Theta(N^2)$ variables enteras y $\Theta(N)$ restricciones (ignorando cotas).

El tercer modelo es un PL Mixto con $m=\Theta(N^2)$ variables continuas, $N$ variables enteras y $\Theta(N^2)$ restricciones.

El cuarto modelo es un PL puro con  $m=\Theta(N^2)$ variables continuas  y $\Theta(2^N)$ restricciones. Un teorema de Edmonds garantiza que los puntos extremos de (M4) son exactamente matchings perfectos.



# Ejercicio 1. Responder en Markdown.

En clases vimos que los modelos 1 y 2 son exactos. El modelo 4 también es exacto (en el sentido integral) y con formulación ideal (es decir, los puntos enteros de la formulación corresponden exactamente a matching perfectos y viceversa).

Veamos que el modelo 3 es, en cierto modo, también un modelo (extendido) exacto con buena formulación.

Demuestre las siguientes propiedades del modelo 3 en las siguientes celdas (usando markdown para formatear)

1.1: Para todo  matching perfecto $M$ existen variables $(x, y)$ factibles tal que $M$ está representado por $x$. Explique el rol de la variable $y$. 

1.2: Demuestre que para todo punto $(x,y)$ factible en (M3), $x$ es la indicatriz de un matching perfecto.

1.3: Demuestre que para todo $(x,y)$ factible en (M3), existe un $x'$ entero tal que  $(x',y)$ es factible en (M3) de igual valor objetivo que $(x,y)$. Concluya que todo óptimo de (M3) se puede transformar en un matching perfecto de costo mínimo.


## Solución al ejercicio 1.1

...

## Solución al ejercicio 1.2

...

## Solución al ejercicio 1.3

...

# Ejercicio 2. En Julia.

Escriba una función 
```julia
emparejamiento(num_modelo, N, w) 
return modelo
```

que reciba como entrada:
```julia
num_modelo: entero en {1,2,3,4} que representa si se trabaja con M1, M2, M3 o M4.
N: número de vértices del grafo
w: matriz de pesos
```
y entregue el modelo en juMP/Gurobi asociado al grafo entregado.


Indicación:
Le recomendamos escribir uno de los modelos primero y seguir con los otros ejercicios, para que pueda probarlo. 
Una vez que esté listo, escriba los demás modelos.


In [ ]:
# Escriba su respuesta aquí
# Esconda la salida de Gurobi escribiendo el comando


function emparejamiento(num_modelo, N, w) 
    mimodelo=Model()
    set_optimizer(mimodelo, Gurobi.Optimizer)
    set_optimizer_attributes(mimodelo, "OutputFlag" => 0, "TimeLimit" => 80)
    
    # escriba el resto de la funcion 
    return mimodelo
end
            
    
# las 2 celdas siguientes contiene un ejemplo que puede usarlo para probar su método 

In [ ]:
# Celda para probar el método anterior
N=6
a = [63, 114, 165, 14, 140, 180]
b = [174, 133, 54, 87, 47, 35]
w = distancias(a,b)
num_mod=1 # o el modelo que desee probar

modelo1=emparejamiento(num_mod, 6, w)
optimize!(modelo1)
@show(objective_value(modelo1))
@show(JuMP.simplex_iterations(modelo1))
@show(JuMP.node_count(modelo1));

#Nota: El valor objetivo debería ser alrededor de 213.9016339...



In [ ]:
# Celda para dibujar el matching encontrado
X = zeros(N,N)
for i in 1:N, j in 1:N
    if i<j
    X[i,j]=value(modelo1[:x][i,j])
    end
end
dibuja(a,b,X)


# Ejercicio 3. En Julia

Parte 1. Escriba una función test_matching(N_MAX) que realice lo siguiente.

1. Genere una instancia aleatoria (N, coordX,coordY,w) para cada N par entre 6 y N_MAX 
   donde coordX, coordY son puntos de la grilla 1:200 por 1:200 y w es su matriz de distancia.
   (el comando sample(1:200, N, replace = false) entrega N enteros entre 1 y 200 sin repetición)
   
   Nota: Guarde las instancias creadas en un arreglo (en el formato que usted desee), durante su ejecución pues debe retornarlas al final. 

2. Para cada instancia (N,coordX,coordY,w) de su lista y cada modelo (num_mod), calcule y guarde los siguientes valores:
    1. Valor de N
    2. valor de num_mod
    3. Numero de variables del modelo (busque el comando en la documentación de JuMP)
    4. Numero de restricciones del modelo (busque el comando en la documentación de JuMP)
    5. Tiempo de creación del modelo a partir de (N,coordX,coordY,w).   
    
            NOTA: Investigue el comando @elapse. Por ejemplo, ejecute
            A=@elapse B = 100*20; A,B; 
    6. Tiempo de resolución del modelo creado.
    7. Valor objetivo encontrado (si logró terminar, si no, entregar 0).
    
            NOTA: Debe testear si el modelo termina de ejecutar usando termination_status
            
    8. Numero de iteraciones de simplex realizadas (JuMP.simplex_iterations(nombre_modelo))
    9. Numero de nodos BnB realizados (JuMP.node_count(nombre_modelo))
   
   Luego elimine integralidad de las variables del modelo, resuelva nuevamente y calcule
   
    10. Tiempo de resolución del modelo fraccional 
    11. Valor objetivo encontrado (si logró terminar, si no, entregar 0)
    12. Numero de iteraciones de simplex realizadas
    
3. Retorne un par INSTANCIAS, TABLA 

Parte 2. Cuando termine su función ejecute test_matching(8) y test_matching(20), mostrando la TABLA creada. 
NOTA: la última tabla debería tener 80 filas.

Parte 3. (opcional) Si su computador logra tiempos razonables sin quedarse sin memoria, ejecute test con el número más alto que pueda correr. 

Sugerencia: Un formato razonable de tabla es el de la siguiente celda


In [ ]:
ENV["COLUMNS"]=200
df=DataFrame(
    :N                 => Int64[],
    :num_modelo        => Int64[],
    :variables         => Int64[],
    :restricciones     => Int64[],
    :t_creacion_ent    => Float64[],
    :t_resolucion_ent  => Float64[],
    :valor_ent         => Float64[],
    :simplex_ent       => Int64[],
    :nodos_bnb         => Int64[],
    :t_resolucion_frac => Float64[],
    :valor_frac        => Float64[],
    :simplex_frac      => Int64[]) 
push!(df,[6, 1, 18, 60, 0.55, 1.223, 14.4, 10, 7, 2, 0.33, 5])

In [ ]:
# Escriba su función acá

function test_matching(N_MAX)
    ...
    return INSTANCIAS, TABLA 
end
    

In [ ]:
# Pruebe su función acá
INSTANCIAS8, TABLA8 = test_matching(8);
TABLA8

In [ ]:
# Pruebe su función acá
INSTANCIAS20, TABLA20 = test_matching(20);
TABLA20

In [ ]:
# Pruebe su función acá con el valor más grande que pueda.


# Ejercicio 4. En Julia y en Markdown

Se sabe que los modelos 1 al 3 requieren de sus condiciones de integralidad.
Busque, ya sea en las instancias generadas al azar de la parte anterior, o en otras que genere usted 
una instancia donde las versiones fraccionales de los 4 modelos sean lo más distintas posibles (en valor objetivo)

Muestre los valores objetivos y dibuje los puntos fraccionales (solo la parte que corresponde a x) optimos asociados a los 4 modelos.

Además, responda a  las siguientes preguntas abiertas:
¿Puede encontrar una instancia donde los 4 valores difieran? (Justifique)
¿Qué modelo tiene el menor valor objetivo, el segundo, el tercero y el cuarto? ¿Deben siempre estar en este orden? (Justifique su respuesta) 

In [ ]:
# instancia encontrada (escribala aquí, debe ser fija, no generada aleatoriamente cada vez que corra el código)
N=
coordx=
coordy=

# Resuelva la instancia con los 4 modelos (recuerde quitar la integralidad)
..

# Muestre una tabla con valores objetivos
..

In [ ]:
#dibujos para las 4 soluciones fraccionales encontradas
..

Respuesta a las preguntas abiertas.
..
